In [ ]:
# default_exp data.transforms

In [ ]:
# hide 
import sys
sys.path.append('..')
import pandas as pd

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
#export
from fastai2.torch_basics import *
from fastai2.data.all import *
# from pyts.image import GramianAngularField, MarkovTransitionField, RecurrencePlot

In [ ]:
from nbdev.showdoc import *

# Transforms

> Basic function to process time-series data before assembling it in a `DataLoaders`.

In [ ]:

class NormalizeTS(Transform):
    "Normalize the Time-Series."
    order = 2
    def setups(self, dsets): 
        print([o[0] for o in dsets])
        self.means,self.stds = 50,2
    def encodes(self, to): 
        return (to-self.means) / self.stds
    def decodes(self, to): 
        return (to*self.stds ) + self.means
        

In [ ]:
from fastseq.data.load import *
horizon,lookback = 2,5
ints = L(np.arange(7)[None,:], np.arange(100,107)[None,:]).map(tensor)

dl = TSDataLoader(ints, horizon = horizon, lookback = lookback, bs=2, after_item=NormalizeTS())
for o in dl:
    print(o)
  
# x = np.array([0,1,2,3,4])
# m,s = x.mean(),x.std()
# test_eq(norm.means['a'], m)
# test_close(norm.stds['a'], s)
# test_close(to['a'].values, (x-m)/s)

[TSTensorSeq([[[  0,   1,   2,   3,   4]],

        [[100, 101, 102, 103, 104]]])]
(TSTensorSeq([[[-25, -24, -24, -23, -23]],

        [[ 25,  25,  26,  26,  27]]]), TSTensorSeqy([[[-22, -22]],

        [[ 27,  28]]]))


In [ ]:
# hide
# TODO normalize proc
def norm_items(self, items, norm):
    items = items.map(tensor)
    r=L()
    for i,ts in enumerate(items):
        ts = ts.float()
        if norm:
            ts = (ts - torch.mean(ts.float(), -1, keepdim = True))/(torch.std(ts.float(), -1, keepdim = True)+1e-8)
        r.append(ts)
    return r

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.transforms.ipynb.
Converted 04_metrics.ipynb.
Converted 05_models.nbeats.ipynb.
Converted 07_interpret.ipynb.
Converted 100_readme_alt.ipynb.
Converted 21_data.loadpandas.ipynb.
Converted index.ipynb.
